# Propozycje PPO

1. **Model studenta z użyciem `@dataclass`**
   - Zdefiniuj klasę `Student` z polami: `imie` (str), `nazwisko` (str), `nr_indeksu` (str), `oceny` (list[int]).
   - Dodaj metodę `srednia()` zwracającą średnią ocen oraz `as_dict()` używającą `asdict()`.

2. **Kolekcja figur z klasą abstrakcyjną**
   - Stwórz abstrakcyjną klasę `Figura(ABC)` z metodą `pole()`.
   - Zaimplementuj podklasy `Prostokat(a, b)` i `Trojkat(a, h)`.
   - Napisz funkcję, która przyjmuje listę `Figura` i sumuje ich pola.

3. **Bankowe konto z enkapsulacją i walidacją**
   - Klasa `Konto` przechowuje stan `_saldo`.
   - Dodaj `@property saldo` oraz setter, który blokuje ustawienie ujemnego salda (ValueError).
   - Dodaj metody `wplata(kwota)` i `wyplata(kwota)` z odpowiednią walidacją i wyjątkiem `InsufficientFunds`.

4. **Dekorator logujący wywołania metod**
   - Zaimplementuj dekorator `@loguj`, który przed i po wykonaniu metody wypisuje jej nazwę i przekazane argumenty.
   - Użyj go na kilku metodach własnej klasy (np. `Kalkulator.add`, `Kalkulator.mul`).

5. **Rejestr klas przez metaklasę**
   - Stwórz metaklasę `RegistryMeta`, która w polu klasowym `registry` będzie trzymać mapę nazwa→klasa dla każdej nowej klasy, która dziedziczy po `Rejestrowana`.
   - Przetestuj, że `RegistryMeta.registry` zawiera wszystkie podklasy.

6. **Wektor z metodami specjalnymi**
   - Klasa `Wektor(x, y)` zdefiniuj `__add__`, `__sub__`, `__mul__` (skalarem), `__eq__`, `__repr__` i `__len__` (długość wektora).
   - Napisz kilka asercji sprawdzających poprawność operacji.

7. **Menadżer kontekstu do pliku JSON**
   - Napisz klasę `JsonFile(path)` implementującą `__enter__`/`__exit__`, która wchłania i zapisuje słownik do pliku jako JSON.
   - W bloku `with` pozwól modyfikować słownik, a po `__exit__` automatycznie zapisz go na dysk.

8. **Kompozycja vs dziedziczenie – samochód i silnik**
   - Zaimplementuj klasę `Silnik` z metodą `start()`.
   - Stwórz `SamochodDziedziczony(Silnik)` i `SamochodZKompozycja`, który w atrybucie `silnik` przechowuje obiekt `Silnik`.
   - Sprawdź, jakie zachowanie (i dlaczego) ma `samochod.start()`.

9. **Własne wyjątki i hierarchia błędów**
   - Zdefiniuj podklasy wyjątków: `BladBazowy(Exception)`, `BladA(BladBazowy)`, `BladB(BladBazowy)`.
   - Napisz funkcję, która losuje wyjątek i w `try/except` obsługuje najpierw `BladA`, potem `BladBazowy`.

10. **Mixin do serializacji obiektów**
    - Stwórz klasę-mixin `JsonSerializable`, która dostarcza metodę `to_json()` (korzystając z `json.dumps(self.__dict__)`).
    - Połącz ją z dowolną klasą biznesową (np. `Produkt`, `Student`) i sprawdź, że możesz łatwo serializować i deserializować jej instancje.


In [1]:
# 1. Model studenta z użyciem @dataclass
from dataclasses import dataclass, field, asdict

@dataclass
class Student:
    imie: str
    nazwisko: str
    nr_indeksu: str
    oceny: list[int] = field(default_factory=list)

    def srednia(self) -> float:
        return sum(self.oceny) / len(self.oceny) if self.oceny else 0.0

    def as_dict(self) -> dict:
        return asdict(self)

# Test
student = Student("Jan", "Kowalski", "S1234", [4,5,3,5])
print("Średnia ocen:", student.srednia())
print("Słownik studenta:", student.as_dict())


Średnia ocen: 4.25
Słownik studenta: {'imie': 'Jan', 'nazwisko': 'Kowalski', 'nr_indeksu': 'S1234', 'oceny': [4, 5, 3, 5]}


In [2]:
# 2. Kolekcja figur z klasą abstrakcyjną
from abc import ABC, abstractmethod

class Figura(ABC):
    @abstractmethod
    def pole(self) -> float:
        pass

class Prostokat(Figura):
    def __init__(self, a: float, b: float):
        self.a = a    # bok a
        self.b = b    # bok b
    def pole(self) -> float:
        return self.a * self.b

class Trojkat(Figura):
    def __init__(self, a: float, h: float):
        self.a = a    # podstawa
        self.h = h    # wysokość
    def pole(self) -> float:
        return 0.5 * self.a * self.h

def suma_pol(figury: list[Figura]) -> float:
    return sum(f.pole() for f in figury)

# Test
figury = [Prostokat(3,4), Trojkat(3,5)]
print("Sumaryczne pole:", suma_pol(figury))


Sumaryczne pole: 19.5


In [3]:
# 3. Konto bankowe z enkapsulacją i własnym wyjątkiem
class BrakSrodkow(Exception):
    """Wyjątek rzucany przy braku środków na koncie."""
    pass

class Konto:
    def __init__(self, saldo_poczatkowe: float = 0.0):
        self._saldo = saldo_poczatkowe  # chronione saldo

    @property
    def saldo(self) -> float:
        return self._saldo

    @saldo.setter
    def saldo(self, kwota: float):
        if kwota < 0:
            raise ValueError("Saldo nie może być ujemne")
        self._saldo = kwota

    def wplata(self, kwota: float):
        if kwota <= 0:
            raise ValueError("Kwota wpłaty musi być dodatnia")
        self._saldo += kwota

    def wyplata(self, kwota: float):
        if kwota <= 0:
            raise ValueError("Kwota wypłaty musi być dodatnia")
        if kwota > self._saldo:
            raise BrakSrodkow("Brak wystarczających środków")
        self._saldo -= kwota

# Test
konto = Konto(100.0)
konto.wplata(50.0)
print("Saldo po wpłacie:", konto.saldo)
try:
    konto.wyplata(200.0)
except BrakSrodkow as e:
    print("Błąd wypłaty:", e)


Saldo po wpłacie: 150.0
Błąd wypłaty: Brak wystarczających środków


In [4]:
# 4. Dekorator logujący wywołania metod
from functools import wraps

def loguj(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Wywołanie {func.__name__} z args={args}, kwargs={kwargs}")
        wynik = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} zwróciło {wynik}")
        return wynik
    return wrapper

class Kalkulator:
    @loguj
    def dodaj(self, a: float, b: float) -> float:
        return a + b

    @loguj
    def mul(self, a: float, b: float) -> float:
        return a * b

# Test
k = Kalkulator()
k.dodaj(2,3)
k.mul(4,5)


[LOG] Wywołanie dodaj z args=(<__main__.Kalkulator object at 0x000002EC6983DD30>, 2, 3), kwargs={}
[LOG] dodaj zwróciło 5
[LOG] Wywołanie mul z args=(<__main__.Kalkulator object at 0x000002EC6983DD30>, 4, 5), kwargs={}
[LOG] mul zwróciło 20


20

In [5]:
# 5. Rejestr klas przez metaklasę
class RegistryMeta(type):
    registry = {}  # mapa nazwa→klasa

    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        if name != "Rejestrowana":
            mcs.registry[name] = cls
        return cls

    @classmethod
    def pokaz_rejestr(mcs):
        return mcs.registry

class Rejestrowana(metaclass=RegistryMeta):
    """Klasa bazowa dla rejestracji podklas."""
    pass

class A(Rejestrowana): pass
class B(Rejestrowana): pass

# Test
print("Zarejestrowane klasy:", RegistryMeta.pokaz_rejestr())


Zarejestrowane klasy: {'A': <class '__main__.A'>, 'B': <class '__main__.B'>}


In [6]:
# 6. Wektor z metodami specjalnymi
class Wektor:
    def __init__(self, x: float, y: float):
        self.x = x  # współrzędna x
        self.y = y  # współrzędna y

    def __add__(self, other):
        return Wektor(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Wektor(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar: float):
        return Wektor(self.x * scalar, self.y * scalar)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return f"Wektor(x={self.x}, y={self.y})"

    def __len__(self):
        return int((self.x**2 + self.y**2)**0.5)

# Test
v1 = Wektor(2,3)
v2 = Wektor(1,1)
print("Dodawanie:", v1 + v2)
print("Odejmowanie:", v1 - v2)
print("Mnożenie:", v1 * 3)
print("Długość v1:", len(v1))
print("Porównanie:", v1 == Wektor(2,3))


Dodawanie: Wektor(x=3, y=4)
Odejmowanie: Wektor(x=1, y=2)
Mnożenie: Wektor(x=6, y=9)
Długość v1: 3
Porównanie: True


In [7]:
# 7. Menedżer kontekstu do pliku JSON
import json

class JsonFile:
    def __init__(self, sciezka: str):
        self.sciezka = sciezka  # ścieżka do pliku

    def __enter__(self):
        try:
            with open(self.sciezka, 'r', encoding='utf-8') as f:
                self.data = json.load(f)  # wczytaj istniejący JSON
        except (FileNotFoundError, json.JSONDecodeError):
            self.data = {}  # pusty słownik jeśli plik nie istnieje
        return self.data

    def __exit__(self, exc_type, exc_val, exc_tb):
        with open(self.sciezka, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=4)

# Test
with JsonFile("dane.json") as dane:
    dane["nowy_klucz"] = "wartość"
    dane["lista"] = [1,2,3]
print("Zapisano dane do dane.json")


Zapisano dane do dane.json


In [8]:
# 8. Kompozycja vs dziedziczenie – samochód i silnik

# Dziedziczenie
class Silnik:
    def start(self):
        print("Silnik uruchomiony")

class SamochodDziedziczony(Silnik):
    """Samochód dziedziczący po Silnik"""
    pass

# Kompozycja
class SamochodZKompozycja:
    def __init__(self):
        self.silnik = Silnik()  # Samochód ma silnik

    def start(self):
        self.silnik.start()

# Test
print("Dziedziczenie:")
sam1 = SamochodDziedziczony()
sam1.start()

print("Kompozycja:")
sam2 = SamochodZKompozycja()
sam2.start()


Dziedziczenie:
Silnik uruchomiony
Kompozycja:
Silnik uruchomiony


In [9]:
# 9. Własne wyjątki i hierarchia błędów
import random

class BladBazowy(Exception):
    """Bazowa klasa wyjątków."""
    pass

class BladA(BladBazowy):
    """Wyjątek specyficzny A."""
    pass

class BladB(BladBazowy):
    """Wyjątek specyficzny B."""
    pass

def losuj_wyjatek():
    wybor = random.choice(['A','B','bazowy','brak'])
    if wybor == 'A':
        raise BladA("Wystąpił błąd A")
    elif wybor == 'B':
        raise BladB("Wystąpił błąd B")
    elif wybor == 'bazowy':
        raise BladBazowy("Wystąpił błąd bazowy")
    return "Brak błędu"

# Test
try:
    wynik = losuj_wyjatek()
    print("Wynik:", wynik)
except BladA as e:
    print("Obsłużono BladA:", e)
except BladBazowy as e:
    print("Obsłużono BladBazowy:", e)


Obsłużono BladBazowy: Wystąpił błąd B


In [10]:
# 10. Mixin do serializacji obiektów
import json

class JsonSerializable:
    def to_json(self) -> str:
        """Serializuje atrybuty instancji do JSON."""
        return json.dumps(self.__dict__, ensure_ascii=False)

    @classmethod
    def from_json(cls, json_str: str):
        """Odtwarza instancję z JSON."""
        data = json.loads(json_str)
        inst = cls.__new__(cls)           # pomiń __init__
        inst.__dict__.update(data)        # załaduj atrybuty
        return inst

class Produkt(JsonSerializable):
    def __init__(self, nazwa: str, cena: float):
        self.nazwa = nazwa
        self.cena = cena

# Test
p = Produkt("Telefon", 1999.99)
json_str = p.to_json()
print("JSON:", json_str)
p2 = Produkt.from_json(json_str)
print("Odtworzony obiekt:", p2.__dict__)


JSON: {"nazwa": "Telefon", "cena": 1999.99}
Odtworzony obiekt: {'nazwa': 'Telefon', 'cena': 1999.99}
